In [ ]:
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms.functional as tran_f
import os
import numpy as np
import pandas as pd
import librosa
import os
from PIL import Image
import random

In [ ]:
import sys
# sys.path.append("../pytorch_spectrogram")
sys.path.insert(1, os.path.join(sys.path[0], '..'))
# from dataloader import ravdessLoader
from utils.utils import accuracy, scale_minmax
from models import mobilenet_v2
from torchvision import transforms

In [ ]:
import torch.utils.data as data
import torch.utils.data as data
class ravdessLoader(data.Dataset):
    def __init__(self, test_folder=[12, 24], is_train=True, spec_path='./ravdess_spectrogram2'):
        self.img_path = spec_path
        self.file_list = os.listdir(spec_path)
        self.classID = [int(x.split('-')[2]) - 1 for x in self.file_list]
        # self.spkID = [int(x.split('_')[0].split('-')[-1].split('.png')[0]) for x in self.file_list]
        self.spkID = [int(x.split('-')[-1].split('.png')[0]) for x in self.file_list]
        self.path_list = []
        self.id_list = []

        if is_train:
            self.transform = transforms.Compose([
                transforms.RandomHorizontalFlip(p=0.5),
                # transforms.RandomVerticalFlip(p=0.5),
                AddGaussianNoise(mean=0., std=1., p=0.5),
                transforms.ToTensor(),
            ])
            for i in range(len(self.file_list)):
                
                if self.spkID[i] not in test_folder:
                    self.path_list.append(self.file_list[i])
                    self.id_list.append(self.classID[i])
        else:
            self.transform = transforms.Compose([transforms.ToTensor()])
            for i in range(len(self.file_list)):
                if self.spkID[i] in test_folder:
                    self.path_list.append(self.file_list[i])
                    self.id_list.append(self.classID[i])
        class_id = np.array(self.id_list)

    def __len__(self):
        return len(self.path_list)

    def __getitem__(self, index):
        img_path = os.path.join(self.img_path, self.path_list[index])
        spc = Image.open(img_path)
        label = int(self.id_list[index])
        if self.transform:
            spc = self.transform(spc)
        return spc, label

In [ ]:
device = torch.device('cuda:0')
from models import squeezenet1_0 as network
net = network(pretrained=False).to(device)

net.load_state_dict(torch.load('../weight/epoch_364.pkl')['model_state'])
print('weight loaded.')

test_trans = transforms.Compose([transforms.ToTensor()])
val_loader = DataLoader(ravdessLoader(spec_path='../data/ravdess_spectrogram2', test_folder=[11, 12, 23, 24], is_train=False), 
            batch_size=1, shuffle=False, num_workers=0) 

print('############# validation {} #############'.format(len(val_loader)))
net.eval()
acc = 0
dic = {
      '0': [0, 0, 0, 0, 0, 0, 0, 0], 
      '1': [0, 0, 0, 0, 0, 0, 0, 0],
      '2': [0, 0, 0, 0, 0, 0, 0, 0],
      '3': [0, 0, 0, 0, 0, 0, 0, 0],
      '4': [0, 0, 0, 0, 0, 0, 0, 0],
      '5': [0, 0, 0, 0, 0, 0, 0, 0],
      '6': [0, 0, 0, 0, 0, 0, 0, 0],
      '7': [0, 0, 0, 0, 0, 0, 0, 0]}
for batch_id, data in enumerate(val_loader):
    spc, label = data
    spc = spc.to(device)
    label = label.to(device)
    pred = net(spc)
    
    classid = pred.max(1, keepdim=True)[1][0][0]
    dic[str(label.item())][classid] += 1
    
    acc += accuracy(pred, label)
    
acc = acc / len(val_loader)
print(acc)
print(dic)

In [ ]:
import seaborn as sns


df = pd.DataFrame(dic)
sns.heatmap(df, linewidths=.25, cmap='YlGnBu', annot=True)